$\newcommand{\xv}{\mathbf{x}}
 \newcommand{\wv}{\mathbf{w}}
 \newcommand{\yv}{\mathbf{y}}
 \newcommand{\zv}{\mathbf{z}}
 \newcommand{\Chi}{\mathcal{X}}
 \newcommand{\R}{\rm I\!R}
 \newcommand{\sign}{\text{sign}}
 \newcommand{\Tm}{\mathbf{T}}
 \newcommand{\Xm}{\mathbf{X}}
 \newcommand{\Zm}{\mathbf{Z}}
 \newcommand{\I}{\mathbf{I}}
 \newcommand{\muv}{\boldsymbol\mu}
 \newcommand{\Sigmav}{\boldsymbol\Sigma}
$



# Markov Decision Processes

<br/>
<br/><br/><br/>

### ITCS 4156

### Minwoo "Jake" Lee

# Goal

The goal of this activity is to define an MDP for a the discrete marble task and then use dynamic programming to solve it. Follow the TODO titles and comments to finish the activity!

# Agenda

1. Discrete Marble Task
    1. Environment and Actions
2. DMarble Environment Class
3. Policy Iteration
    1. Notation
    2. Algorithm Summary

In [1]:
# Common imports
import numpy as np
from copy import deepcopy as copy

# To plot pretty figures
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

import IPython.display as ipd  # for display and clear_output
from IPython.core.debugger import set_trace

# print only 3 digits under decimal point
np.set_printoptions(precision=3, suppress=True)

# Discrete Marble Task 

The discrete marble task is actually a rather simple task. The goal of this task is to move a marble from any given location on a 1-D line to the goal location. The marble is restricted to discrete actions of move left, right, or don't move (no action).

## Environment and Actions
For this task our environment is defined by a marble on a 1-D line. This "1-D line" also defines our possibles actions as the following: move left, move right, or don't move (no action). Each action is defined as a discrete number: -1 (left), 0 (don't move), or 1 (right). These discrete numbers also determine the distance the marble moves on the 1-D line. This means each state must be defined by an integer! In our case, our state representations will range between 0 and 10 (see below image). At some point on the 1-D line there is also a goal state. Thus, the goal of the task is to explore the environment to find the goal state.

If you're still confused on the environment setup see the picture below. Here you can see the marble in red with its 3 possible actions. Each move left or right actions moves the marble 1 unit (left or right) on the 1-D line. The goal state location is at 5 and the starting marble state location is at 3. Thus, the marble has to perform 2 move right actions to reach the goal state. 

<img src="https://i.imgur.com/MwXDZPI.png">



To start off, we must define the MDP before we can solve the discrete marble task. Let's define what our representations will be for the standard MDP model $(S, A, P, R, \gamma)$.


# TODO:

Answer the below questions or fill in the blanks (fill in the blanks are indicated by the **[blank]** symbol). Add your answer in the markdown cell underneath the question. Note, these should be plain text answers, no coding is needed.

**TODO (1)**

What are all the possible states $S$ the marble could be in?

**TODO (1) ANSWER**

$S$ consists of states $0$ through $10$, inclusive

**TODO (2)**

The three actions $A$ are **[blank]** and they are represented by the three integers **[blank]**.

**TODO (2) ANSWER**

Movement to the left and right (and the lack thereof) and they are represented by $\{-1,0,1\}$

**TODO (3)**

The transition probabilities $T$ in the environment are deterministic, so all transition probabilities are **[blank]**. 

**TODO (3) ANSWER**

static, and known in advance. (I think this is what the question was aiming for?)

In our case, the probabilities for a transition of states are equally distributed across all three actions, meaning each action has a $0.33$ chance of occuring.

**TODO (4)**

The reward $R$ is given at the end of the task when the marble reaches the goal. Thus, we can say $R$ equals to **[blank]** when the marble reaches goal, otherwise $R$ is equal to **[blank]**. Hint, our reward function $R$ is binary!

**TODO (4) ANSWER**

$R = 1$ when at the goal state, otherwise $R = 0$

# DMarble Environment Class

Now it's time to implement the discrete marble task! Below we have defined a class called `DMarble` that defines the discrete marble task. Be sure to read through this class and understand what each method is doing, as we'll be using them later. There are a few TODOs you need complete within `DMarble` class as well.

### General Python Note: Private Methods and Variables
Note, that any method/variable that has a leading underscore `_` indicates that said method/variable is private. Python doesn't have true private methods/variables like other languages, rather it has pseudo private methods/variables. This means you can still access any private method/variable marked with a leading underscore. However, the leading underscore indicates to users that the method/variable should be treated as if it was a true private method/variable (meaning, the user should not call the method/variable from outside the class or things might go wrong later on).

For more information see this GeeksforGeeks [post](https://www.geeksforgeeks.org/private-variables-python/).

### TODO
#### init() method
1. If `start` is None, not given, then set our start position `cur_state` to be random state between our state representation bounds, given by the class variable `self.bounds`. Remember, our lowest possible state representation is 0 and the highest possible state representation is 10.
    1. Hint: Use [`np.random.randint()` function](https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.random.randint.html) to generate a random integer between two bounds. Be sure to add 1 to the end bound since the upper bound of `np.random.randint()` is  exclusive!
2. Else, if `start` is not `None`, is given, then set `cur_state` equal to `state`.

#### next() method
3. Generate our next state `s1` by adding the input action `a` to the input state `s`. Remember, our state representations are integers!

#### _get_reward() method
4. Generate the reward function by creating an if statement to check if the next state input  is the goal state.That is, if `s1` is equal `self.goal` then return a reward of 1, else return a reward of 0.
    1. Hint: Python let's you write one line if-else statements for short if statements like the above. Check out this [stack overflow post](https://stackoverflow.com/questions/2802726/putting-a-simple-if-then-else-statement-on-one-line) for how one line if-else statements work.

In [2]:
class DMarble(object):
    """ Reinforcement learning Model for Discrete Marble. 
    
        states: x, dx
        action: action [-1, 0, 1]

        |            ___                     |
        |___________|///|____G_______________|
                    <- ->    
    """

    def __init__(self, goal=5, **params):
        """  create an object with the given goal
        
            attributes
            ===========
            act_bound   list
                        the bounds for actions 
            bound       list
                        the bounds for states 
            goal        list
                        the goal location
        """
        self.act_bound = [-1, 1]
        self.bound = [0, 10]
        self.goal = goal

    def init(self, start=None):
        """ initialize the starting position
        
            paramters
            ----------
            start   int
                    the start position (optional). if not given, random
            @return current state
        
        """
        
        if start is None:
            # TODO (1)
            cur_state = np.random.randint(0,11)
        else:
            # TODO (2)
            cur_state = start
        return cur_state
       
   
    def next(self, s, a):
        """ simulate 1 step by taking the action from the given state 
            it updates the current states (_cur_state) and 
            flag variable if it ends the episode (_done)
        
            paramters
            ----------
            s       int
                    current state
            a       int
                    action 
            @return list of next state, reward, done
        """
        # TODO (3)
        s1 = s + a
        
        # check if it hits the ends 
        if s1 < self.bound[0]:
            s1 = self.bound[0]
        elif s1 > self.bound[1]:
            s1 = self.bound[1] 
        r1 = self._get_reward(s1)
        
        return s1, r1
    
    ###### getters #############################
    
    ### reward function 
    def _get_reward(self, s1):
        # TODO (4)
        return 1 if s1 == self.goal else 0
    
    ### function to check the end condition
    def is_done(self, s):
        return s == self.goal

    ### get all the possible actions
    def get_actions(self):
        # return all the possible actions in a list
        return [-1, 0, 1]
    
    ### get a random action
    def get_random_action(self):
        # return randomly selected action 
        return np.random.choice(self.get_actions())
    
    ### get the number of states
    def get_n_states(self):
        return self.bound[1] - self.bound[0] + 1
    
    ### get the number of actions
    def get_n_actions(self):
        return len(self.get_actions())

Let's create an object from the DMarble class and simulate/test if it works. By default, our goal location is set to 5 if we do not pass a value for the `goal` parameter. Regardless, below we are explicitly setting the goal location to 5 for ease of reading!

In [3]:
env = DMarble(goal=5)

### TODO: 
 Before we solve this task let's manually interact with the environment. To do so, let's manually input the start location and actions until we reach the goal state.
 
1. Set the initial start location of the marble to 3 using the `.init()` method. Store the output into `s`. 
2. Now apply sequence of actions -1, 1, 0, 1, 1 and observe states the marble enters. Store every output into `s` and `r`.
    1. Take a move left action (-1) 
    2. Take a move right action (1) 
    3. Take a no action (0)
    4. Take a move right action (1) action
    5. Take a move right action (1) action
  
3. Check if we have reached the goal state using the `.is_done()` method. Store the output into `reached_goal`.
    1. Hint: You must pass the current state `s` to this method!

In [4]:
# TODO (1)
s = env.init(3)
s

3

In [5]:
# TODO (2A)
s, r = env.next(s, -1)
s

2

In [6]:
# TODO (2B)
s, r = env.next(s, 1)
s

3

In [7]:
# TODO (2C)
s, r = env.next(s, 0)
s

3

In [8]:
# TODO (2D)
s, r = env.next(s, 1)
s

4

In [9]:
# TODO (2E)
s, r = env.next(s, 1)
s

5

In [11]:
# TODO (3)
reached_goal = env.is_done(s)
print("Have we reached the goal? {}".format(reached_goal))
assert reached_goal == True 

Have we reached the goal? True


# Policy Iteration

Now, let's implement the policy iteration algorithm to solve the discrete marble task. To remind you, here is the copy of the class notes and some pseudo code to summarize the algorithm. 


![](http://incompleteideas.net/sutton/book/first/4/img158.gif)

## Notation
- S: a finite set of states
- A: a finite set of actions
- P: a state transition probability
    - $P^a_{ss′}= P[S_{t+1}=s′|S_t=s,A_t=a]$
- R: a reward function
- γ: a discount factor (gamma)

## Algorithm Summary
* Start with an initial policy $\pi^0$.
* Iteratively,
    * Evaluate policy ($V^n(x) = V^{\pi^n}(x)$):
    $$ V^n(s) = R(s, a = \pi^n(s)) + \gamma \sum_{s^\prime} P(s^\prime | s, a = \pi^n(s)) V^{n}(s^\prime) $$
    * Improve policy:
    $$ \pi_{n+1}(s) = \arg \max_a \Big[ R(s, a) + \gamma \sum_{s^\prime} P(s^\prime | s, a) V^{n}(s^\prime) \Big] $$
    
* Stop condition:
    * Policy does not change anymore (for about 10 iterations)
    * The changes in evaluation of values are minor 
 
The psuedo code is located in the below image. Note, $P(s^\prime | s, a) $ above and $P(s^\prime, r | s, a)$ in the below image can be treated as the same thing. You'll also notice the equations are written slightly different, either form is okay.
![](https://miro.medium.com/max/2952/1*ZYq8bX-q4Z8-8aPndQaz9w.png)

### TODO:

Implement the policy iteration algorithm here. Here, our goal state is located at state 5, since we are reusing the same `env` object as before, and our starting state will be randomized (really we'll be treating every state as a starting state). Read the code and fill in the TODOs.

#### Policy Evaluation
1. Execute the current action using the `.next()` method by pass current state `s` and action. The current action is determined by indexing `As` (the set of all actions) at `a` (the current action index of `As`). Store the new state and reward into `s1` and `r1`.

2. Compute the value of the given state by computing the Policy Evaluation equation $r + \gamma V(s')$. To do so multiple, `gamma` by the next state value (done by indexing `V` at `s1`), then add our reward for entering the next state `r1`. Remember, our state transition probability is 1, since the state transition $P(s^\prime | s, a)$ is deterministic (we can only go to one other state from any given state)! Store output into `val`.


#### Policy Improvement
3. Execute the current action using the `.next()` method by pass current state `s` and action. The current action is determined by indexing `As` (the set of all actions) at `a` (the current action index of `As`). Store the new state and reward into `s1` and `r1`.

4. Compute the value of the given state by computing the Policy Improvement equation $r + \gamma V(s')$. To do so multiple, `gamma` by the next state value (done by indexing `V` at `s1`), then add our reward for entering the next state `r1`. Store the output into `val`

5. Pick the action with the highest probability by taking the `np.argmax()` of `values`. Store the output into `new_a`.
    1. Note: `values` is a 1-d list so no axis parameter is needed for `np.argmax()`

In [14]:
import pandas as pd
from IPython.display import display, HTML

# Defin discounting factor
gamma = 0.9

# Define threshold for convergence
thresh = 0.001

# Reinitialize the environment
env.init()
As = env.get_actions() # all the actions

# Get number of states and actions
n_s = env.get_n_states()
n_a = env.get_n_actions()

# Define state value table
V = np.zeros(env.get_n_states())

# Initialize a stochastic policy so that our algorithm will explore
# all actions for each state during the first iteration. Doing so
# means we have to add an action probability to our Policy Evaluation equation!
pi = np.ones((n_s, n_a)) / n_a

# Build fancy Pandas DataFrames for visualizing initial state value 
# array (V) and policy array (pi).
V_columns = [ "state {}".format(s) for s in np.arange(0, 11)]
V_df = pd.DataFrame(V.reshape(1, -1), columns=V_columns)
V_df.index = ['values']

pi_columns = ['Left', "Don't Move", 'Right']
pi_rows = ["state {}".format(s) for s in np.arange(0, 11)]
pi_df = pd.DataFrame(pi, columns=pi_columns)
pi_df.index = pi_rows

print ("Initialzed Values")
print("Starting state values:")
display(HTML(V_df .to_html()))
print("Starting stochastic policy probabilities:")
display(HTML(pi_df.to_html()))
print("\n\n\n")

# Policy Iteration 
i = 0
while True:
    print ("Iteration #{}".format(i+1))
    i += 1
    
    ############################################
    # Policy Evaluation (ONLY updating our state values V)
    converged = False
    while not converged:
        delta = 0
        
        # Evaluate for all states
        for s in range(n_s):
            v = V[s] # store previous state values
            V[s] = 0 # reset state value so new value can be calculated
            
            # Evaluate for all actions given a state for the initial pi (stochastic).
            # Once we apply our first policy improvement then we are only updating 
            # a signle deterministic action.
            for a in range(n_a):
                # TODO (1):
                s1, r1 = env.next(s, As[a])
                # TODO (2): action probability * (reward + discounting factor * V')
                val = r1 + gamma * V[s1]
                V[s] += pi[s, a] * val
                
            # Compute difference between previous and current state values
            delta = max(delta, abs(V[s] - v))
            
        # Check if the difference in our previous and current state values is
        # smaller than the defined threshold for convergence. 
        # If so, then our state values have converged for this iteration!
        if delta < thresh:
            converged = True
            
        print(V)        
    print('\n')
    print("Policy before Policy Improvement:\n {}".format(pi))
    ############################################
    # Policy Improvement (ONLY updating our policy pi USING V)
    stable = True
    for s in range(n_s):
        old_a = np.argmax(pi[s])  # store current policy
        
        values = []
        # Compute the values for each action for a given state
        for a in range(n_a):
            # TODO (3)
            s1, r1 = env.next(s, As[a])
            # TODO (4)
            val = r1 + gamma * V[s1]
            values.append(val)
        
        # TODO (5)
        new_a = np.argmax(values)
        
        # If the old action does not equal the new action 
        # then we have not stablized yet.
        if old_a != new_a:
            stable = False
        
        # Update the policy to a deterministic policy.
        # Meaning, one action has a value of 1 while
        # the rest have 0 as their value.
        pi[s] = np.eye(n_a)[new_a]
    
    print("Policy after Policy Improvement:\n {}".format(pi))
    print("\n\n\n")
    if stable:
        print("Policy is stable now.")
        break

Initialzed Values
Starting state values:


,state 0,state 1,state 2,state 3,state 4,state 5,state 6,state 7,state 8,state 9,state 10
values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Starting stochastic policy probabilities:


,Left,Don't Move,Right
state 0,0.333333,0.333333,0.333333
state 1,0.333333,0.333333,0.333333
state 2,0.333333,0.333333,0.333333
state 3,0.333333,0.333333,0.333333
state 4,0.333333,0.333333,0.333333
state 5,0.333333,0.333333,0.333333
state 6,0.333333,0.333333,0.333333
state 7,0.333333,0.333333,0.333333
state 8,0.333333,0.333333,0.333333
state 9,0.333333,0.333333,0.333333






Iteration #1
[0.    0.    0.    0.    0.333 0.463 0.614 0.239 0.093 0.036 0.018]
[0.    0.    0.    0.1   0.511 0.717 0.785 0.334 0.141 0.061 0.031]
[0.    0.    0.03  0.165 0.613 0.808 0.849 0.373 0.164 0.073 0.037]
[0.    0.009 0.053 0.205 0.655 0.844 0.874 0.39  0.174 0.079 0.04 ]
[0.003 0.017 0.068 0.223 0.673 0.858 0.885 0.397 0.179 0.082 0.041]
[0.005 0.022 0.076 0.232 0.681 0.864 0.89  0.401 0.181 0.083 0.042]
[0.007 0.025 0.079 0.235 0.684 0.867 0.892 0.402 0.182 0.083 0.042]
[0.008 0.027 0.081 0.237 0.686 0.868 0.893 0.403 0.182 0.084 0.042]
[0.008 0.027 0.082 0.238 0.687 0.869 0.893 0.403 0.182 0.084 0.042]


Policy before Policy Improvement:
 [[0.333 0.333 0.333]
 [0.333 0.333 0.333]
 [0.333 0.333 0.333]
 [0.333 0.333 0.333]
 [0.333 0.333 0.333]
 [0.333 0.333 0.333]
 [0.333 0.333 0.333]
 [0.333 0.333 0.333]
 [0.333 0.333 0.333]
 [0.333 0.333 0.333]
 [0.333 0.333 0.333]]
Policy after Policy Improvement:
 [[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 

As the task is rather simple, the algorithm can quickly find the solution. The printouts show how the state values are updated until convergence. The state value convergence looks reasonable, as you can see the state values increase as the states get closer to the goal (the goal is at state 5, so the state values increase as they approach it).

We can also see how our stochastically initialized policy changes to a deterministic policy by looking at the before and after policy improvement printouts. In addition, we can see our policy quickly converges. However, the algorithm doesn't stop until the second iteration, where we can finally assert that our before and after policy improvement policies don't change.

If we print the our final policy $\pi$, we can see the optimal actions to take to reach the goal node.

In [15]:
pi_columns = ['Left', "Don't Move", 'Right']
pi_rows = ["state {}".format(s) for s in np.arange(0, 11)]
pi_df = pd.DataFrame(pi, columns=pi_columns, dtype=int)
pi_df.index = pi_rows
pi_df

,Left,Don't Move,Right
state 0,0,0,1
state 1,0,0,1
state 2,0,0,1
state 3,0,0,1
state 4,0,0,1
state 5,0,1,0
state 6,1,0,0
state 7,1,0,0
state 8,1,0,0
state 9,1,0,0


By taking the argmax of `pi` and indexing our environment's actions `env.get_actions()` we can get the actions the optimal policy would take in each state.

In [16]:
opt_policy_idx = np.argmax(pi, axis=1)
opt_policy = np.array(env.get_actions())[opt_policy_idx]

columns = [ "state {}".format(s) for s in np.arange(0, 11)]
df = pd.DataFrame(opt_policy.reshape(1, -1), columns=columns)
df.index = ['aciton values']
df

,state 0,state 1,state 2,state 3,state 4,state 5,state 6,state 7,state 8,state 9,state 10
aciton values,1,1,1,1,1,0,-1,-1,-1,-1,-1
